In [ ]:
import re, os, sys, nltk
import xml.etree.cElementTree as et
import pickle, base64, time
from heapq import *
import math, operator, json, copy

In [ ]:
stemmer = nltk.stem.SnowballStemmer('english')
stop_words = {}
stop_file = open("stop_words.txt", "r")
words = stop_file.read()
words = words.split(",")
for word in words:
    word = word.strip()
    if word:
        stop_words[word[1:-1]] = 1

In [ ]:
fields = {}
fields["t"] = open("index/title_output.txt","r")
fields["c"] = open("index/category_output.txt","r")
fields["i"] = open("index/infobox_output.txt","r")
fields["d"] = open("index/body_text_output.txt","r")

In [ ]:
words_position = {}
with open("index/word_position.json",'r') as f:
    words_position = json.load(f)

In [ ]:
title_position = pickle.load(open("index/title_position.pickle", "rb"))

In [ ]:
titles = open("index/titles.txt",'r')
results = 10

In [ ]:
while True:
    query_result = []
    query = input("> ")
    if (query == "quit") :
        break
    start = time.time()
    result_doc = {}
    if ":" in query:
        query_fields = query.split(" ")
        for queries in query_fields:
            field,query = queries.split(":")
            if field == "ref" or field == "ext" or field == "body":
                field = "d"
            elif field == "title":
                field = "t"
            elif field == "category":
                field = "c"
            elif field == "infobox":
                field = "i"
            words = query.split(" ")
            for word in words:
                word = stemmer.stem(word)
                if word and word not in stop_words:
                    if word in words_position and field in words_position[word]:
                        docs = []
                        pointer = words_position[word][field]
                        fields[field].seek(pointer)
                        posting_list = fields[field].readline()[: -1]
                        posting_list = posting_list.split(",")
                        for doc in posting_list:
                            doc_no, score = doc.split(":")
                            if doc_no not in result_doc:
                                result_doc[doc_no] = 0
                            result_doc[doc_no] += float(score)
    else:
        
        query_words = query.split(" ")
        for word in query_words:
            word = word.strip()
            word = stemmer.stem(word)
            if word not in stop_words:
                if word in words_position:
                    docs = []
                    for field in words_position[word].keys():
                        pointer = words_position[word][field]
                        fields[field].seek(pointer)
                        posting_list = fields[field].readline()[: -1]
                        posting_list = posting_list.split(",")
                        for doc in posting_list:
                            doc_no, score = doc.split(":")
                            if doc_no not in result_doc:
                                result_doc[doc_no] = 0
                            result_doc[doc_no] += float(score)
        
    count = 1
    result_doc = sorted(result_doc.items(), key = operator.itemgetter(1), reverse = True)
    for document in result_doc :

        position = title_position[int(document[0]) - 1]
        titles.seek(position)
        query_result.append(titles.readline()[: -1])
        if count >= results :
            break
        count += 1
        
    end = time.time()
    print("Time Taken :- ",(end-start))
    if len(query_result) == 0:
        print("No result found")
    else:
        print("Number of Results :- ",len(query_result))
        for result in query_result:
            print(result)
                        